# Example 3: Propagation of H2O vibration under polynomial PES

| run type      | wavefunction | backend | Basis  | steps |
| ---           | ---          | ---     | ---    | ---   |
| propagation | MPS-SM | Numpy   | HO-FBR | 5000    |

## 1. Import modules

- Required in **any** calculations

In [1]:
from pytdscf import BasInfo, Model, Simulator

## 2. Set FBR primitive basis

In [2]:
from math import sqrt

from discvar import PrimBas_HO

from pytdscf import units
from pytdscf.potentials.h2o_potential import k_orig

freqs = [
    sqrt(k_orig[(1, 1)]),
    sqrt(k_orig[(2, 2)]),
    sqrt(k_orig[(3, 3)]),
]  # a.u. (sqrt{omega^2} = omega)
nprims = [9, 9, 9]
prim_info = [
    [
        PrimBas_HO(0.0, omega * units.au_in_cm1, nprim)
        for nprim, omega in zip(nprims, freqs, strict=False)
    ]
]  # number of state is 1 --> S0
nstate = len(prim_info)
ndof = len(prim_info[0])
basinfo = BasInfo(prim_info)

**MPS-MCTDH wavefunction**
$$
|\Psi_{\rm{MPS-MCTDH}}\rangle = \sum_{\mathbf \{j\}}\sum_{\mathbf \{\tau\}}
a\substack{j_1 \\ 1\tau_1}a\substack{j_2 \\ \tau_1\tau_2} \cdots a\substack{j_f \\ \tau_{f-1}1}
|\varphi_{j_1}^{(1)}(q_1)\rangle|\varphi_{j_2}^{(2)}(q_2)\rangle
\cdots|\varphi_{j_f}^{(f)}(q_f)\rangle
$$
where SPF is 
$$
\varphi_{j_p}^{(p)}(q_p) = \sum_{i_p=1}^{n_p} c_{i_p}^{j_p}\chi_{i_p}^{(p)}(q_p) \; (j_p = 1,2,\ldots, N_p)
$$

Here, select $\{\chi_{i_p}^{(p)}(q_p)\}$ as Harmonic Oscillator eigenfunction.
See detail in [documenation](https://qclovers.github.io/PyTDSCF/pytdscf.html#pytdscf.primints_cls.poly_HO_FBR).
Here one define $n_p$ = 9, $N_p$ = 9. (Standard Method)

**NOTE**

- First argument of `Primbas_HO` is displaced dimensionless coordinate from q=0.0.

- In MPS,  $n = N$ (SM) is usually better than $n < M$ (MCTDH).  Only when using a laptop, MCTDH may be better. (RAM required in MCTDH is smaller than SM.)

## 3. Set Hamiltonian (Polynomial Function)

Here, one uses pre-calculated Polyonimal PES and DMS.
And `read_potential_nMR` includes kinetic terms as default.

In [3]:
from pytdscf.hamiltonian_cls import read_potential_nMR

hamiltonian = read_potential_nMR(k_orig)
operators = {"hamiltonian": hamiltonian}

## 4. Set wavefunction (MPS) and All Model

- `m_aux_max` is MPS bond dimension (maximum of auxiliary index $\tau_p$)


In [4]:
model = Model(basinfo, operators)
model.m_aux_max = 9

## 5. Execute Calculation

- time step width is defined by `stepsize`=0.05 fs

In this calculation, one runs

- Real-time propagation

- Restart from $\hat{\mu}|\Psi_{\rm GS}\rangle$ wavefunction. (restart file suffix is `_dipole`)

F.Y.I., See also [documentation](https://qclovers.github.io/PyTDSCF/pytdscf.html#pytdscf.const_cls.Const.set_runtype)


**NOTE**

- Runtype cannnot rebind. If you change runtype, you should restart the kernel.

- JAX is better when simulating more large systems. (f>6, m>10)

- If `AVG Krylov iteration` in the log file is much larger than 5, you should set smaller timestep.

In [5]:
jobname = "h2o_polynomial"
simulator = Simulator(jobname, model, backend="numpy")
simulator.propagate(
    maxstep=10000,
    stepsize=0.05,
    restart=True,
    savefile_ext="_prop",
    loadfile_ext="_dipole",
)  # i.e., 500 fs

2024-12-18 11:25:18,585 - INFO:main.pytdscf._const_cls - 
     ____     __________   .____ ____   _____
    / _  |   /__  __/ _ \ / ___ / _  \ / ___/
   / /_) /_  __/ / / / ||/ /__ / / )_// /__
  /  ___/ / / / / / / / |.__  / |  __/ ___/
 /  /  / /_/ / / / /_/ /___/ /| \_/ / /
/__/   \__, /_/ /_____/_____/ \____/_/
      /____/

2024-12-18 11:25:18,585 - INFO:main.pytdscf._const_cls - Log file is ./h2o_polynomial_prop/main.log
2024-12-18 11:25:18,585 - INFO:main.pytdscf.simulator_cls - Set integral of FBR basis
2024-12-18 11:25:18,601 - INFO:main.pytdscf.simulator_cls - Set initial wave function (FBR basis)
2024-12-18 11:25:18,602 - INFO:main.pytdscf.simulator_cls - Wave function is loaded from wf_h2o_polynomial_dipole.pkl
2024-12-18 11:25:18,608 - INFO:main.pytdscf.simulator_cls - Wave function is saved in wf_h2o_polynomial_prop.pkl
2024-12-18 11:25:18,608 - INFO:main.pytdscf.simulator_cls - Start initial step    0.000 [fs]
2024-12-18 11:25:18,638 - INFO:main.pytdscf.simulator_cls - E

(0.0208961103101235, <pytdscf.wavefunction.WFunc at 0x116c1e480>)

## 6. Check Log file
See `h2o_polynomial_prop/main.log`, which is defined as `jobname`.

In [6]:
!tail h2o_polynomial_prop/main.log

| autocorr:  0.6236-0.7767i| pop 1.0000 | ene[eV]:  0.5686121 | time[fs]:  499.700 | elapsed[sec]:   244.94 
| autocorr:  0.5543-0.8274i| pop 1.0000 | ene[eV]:  0.5686121 | time[fs]:  499.750 | elapsed[sec]:   244.97 
| autocorr:  0.4809-0.8720i| pop 1.0000 | ene[eV]:  0.5686121 | time[fs]:  499.800 | elapsed[sec]:   244.99 
| autocorr:  0.4039-0.9100i| pop 1.0000 | ene[eV]:  0.5686121 | time[fs]:  499.850 | elapsed[sec]:   245.02 
| autocorr:  0.3240-0.9413i| pop 1.0000 | ene[eV]:  0.5686121 | time[fs]:  499.900 | elapsed[sec]:   245.04 
Saved wavefunction  499.950 [fs]
| autocorr:  0.2417-0.9656i| pop 1.0000 | ene[eV]:  0.5686121 | time[fs]:  499.950 | elapsed[sec]:   245.07 
End  9999 step; propagated  499.950 [fs]; AVG Krylov iteration: 5.00
End simulation and save wavefunction
Wave function is saved in wf_h2o_polynomial_prop.pkl


/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/pty.py:95: RuntimeWarning: Using fork() can cause Polars to deadlock in the child process.
In addition, using fork() with Python in general is a recipe for mysterious
deadlocks and crashes.

The most likely reason you are seeing this error is because you are using the
multiprocessing module on Linux, which uses fork() by default. This will be
fixed in Python 3.14. Until then, you want to use the "spawn" context instead.

See https://docs.pola.rs/user-guide/misc/multiprocessing/ for details.

If you really know what your doing, you can silence this warning with the warning module
or by setting POLARS_ALLOW_FORKING_THREAD=1.

  pid, fd = os.forkpty()
